##### Miljön är *LunarLander-v3*

    Action space: diskret (gör inget, vänster motor, huvudmotor, höger motor)
    Observation space: box (skeppets kordinater, hastighet, vinkel, vinkelhastighet, markkontakt per ben)

Löst episod: minst 200 poäng

##### Modeller

DQN
- Använder sig av Q-Learning för att hitta optimala policyn
- Action-values → lär sig vilka handlingar är bra
- Off-policy → lär sig av en annan policy än den agenten använder
- Kan, i princip, använda data från vilken punkt som helst under inlärning för att lära sig

PPO
- Lär sig en policy direkt och försöker optimera den
- State-values → lär sig vilka tillstånd är bra
- On-policy → lär sig på basis av policyn som agenten använder
- Använder endast den senaste datan för att lära sig

Två väldigt olika typer av modeller.
<br>
<br>

Undersökte vad hyperparametrarna påverkar och vilka värden brukar användas för att få en uppfattning om var det lönar sig att börja med hyperparametrarna. Testade med olika kombinationer, samt längder på inlärning och observerade resultatet – evalurade modellerna ofta under träning för att se utvecklingen. För DQN använde jag även optuna för att försöka hitta optimala parametrar, men resultaten var inte värst bra och gick tillbaka till att testa manuellt. 

DQN krävde mer testning och justering av hyperparametrar än PPO för att lösa miljön. PPO fungerade ganska bra med de flesta av hyperparametrarna som default värden och var överlag mindre känslig för möjligen ooptimala(re) hyperparametrar, medans när man gjorde ett sämre hyperparameterval till DQN syntes det tydligt. Samma kunde observeras med överträning – att fortsätta träna PPO ett tag efter att den löst miljön konsekvent verkade inte direkt leda till någon försämring i prestation, medans DQN verkade känsligare för detta.

\*\* länk för tillhörande graf finns under varje kodsnutt, då plotly grafer har tendes att inte synas efter filuppladdning

In [24]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [25]:
import gymnasium as gym
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
import optuna
from stable_baselines3.common.evaluation import evaluate_policy
import os

#### PPO

In [3]:
env_vec = make_vec_env("LunarLander-v3", n_envs=16)
env_eval = make_vec_env("LunarLander-v3", n_envs=5)
eval_callback = EvalCallback(
    env_eval,
    best_model_save_path="./models/ppo_best",
    eval_freq=1000,
    n_eval_episodes=20,
    log_path='./logs/ppo'
)

`learning_rate=1e-4` – Låg learning rate för stabilare inlärning, nya upplevelser har lägre direkt inverkan på det redan lärda
    
`gae_lambda=0.98` – (lägre) bias vs (högre) varians, högre verkade göra inlärningen lite stabilare och modellen bättre.

`n_steps=1024` – Uppdaterar policyn varje 1024 steps –> relativt ofta men detta verkade producera bättre modell än högre värde

`ent_coef=0.01` – Hur mycket agenten utforskar, utforskar lite

`gamma=0.999` – Värderar lånsiktiga belöningar, högre värde verkade fungera bättre

`n_epochs=10` – Hur många gånger samlade datan används för inlärning, högre verkade producera stabilare inlärning speciellt i början, men lägre fungerade också

In [ ]:
model_ppo = PPO(
    "MlpPolicy",
    env_vec,
    learning_rate=1e-4,
    verbose=1,
    gae_lambda=0.98,
    n_steps=1024,
    ent_coef=0.01,
    gamma=0.999,
    n_epochs=10
)

Using cpu device


In [ ]:
model_ppo.learn(total_timesteps=2e6, callback=eval_callback)

#### DQN

In [2]:
env_vec = make_vec_env("LunarLander-v3", n_envs=16)
env_eval = make_vec_env("LunarLander-v3", n_envs=5)
eval_callback = EvalCallback(
    env_eval,
    best_model_save_path="./models/dqn_best",
    eval_freq=1000,
    n_eval_episodes=20,
    log_path='./logs/dqn'
)

/Users/ellenmolin/Desktop/prescriptiveAn/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


`learning_rate=2e-5` – Låg learning rate för stabilare inlärning, nya upplevelser har lägre direkt inverkan på det redan lärda

<br> 

`batch_size=256` – Hur många "minnen" konsulteras vid uppdatering, högre gjorde inlärning betydligt snabbare

`buffer_size=100_000` – "Längden på agentens minne", högre gjorde inlärning betydligt snabbare

- `batch_size=128` och `buffer_size=50_000` producerade även en agent som kunde lösa miljön, men `batch_size=128` och `buffer_size=100_000` var dålig kombination

<br>

`gamma=0.99` – Värderar långsiktiga belöningar, 0.999 verkade producera sämre modell vilket var intressant

`target_update_interval= 500` – Uppdaterar target network varje 500 steps, oftare verkade producera bättre modell

`learning_starts=100` – Observerar miljön i 100 steps före inlärning börjar

`gradient_steps=-1` – Default (1) skapade dålig agent, detta fungerade betydligt bättre

`exploration_fraction=0.12` – Utforskning minskar till 0.02 under första 12% av träning

`exploration_final_eps=0.02` – Utforskar en aning, lågt värde verkade göra agenten stabilare

2st dolda lager i neurala nätverket med storlek 256

In [ ]:
net_arch = [256, 256]
policy_kwargs = dict(net_arch=net_arch)

model_dqn = DQN(
    "MlpPolicy",
    env_vec,
    learning_rate=2e-5,
    batch_size=256,
    buffer_size=100_000,
    verbose=1,
    gamma=0.99,
    target_update_interval= 500,
    learning_starts=100,
    gradient_steps=-1,
    exploration_fraction=0.12,
    exploration_final_eps=0.02,
    policy_kwargs=policy_kwargs
)

Using cpu device


In [ ]:
model_dqn.learn(total_timesteps=9e5, callback=eval_callback)

##### Optuna study

Söker optimala parameterkombinationer, testade olika kombinationer för kategoriska parametrar och värdeintervaller.

In [ ]:
def objective(trial):

    trial_env = gym.make("LunarLander-v3")
    trial_env = Monitor(trial_env)
    
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)
    gamma = trial.suggest_float('gamma', 0.98, 0.9999)
    layer_size = trial.suggest_categorical('layer_size', [64, 128, 256])    
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])
    learning_starts = trial.suggest_categorical('learning_starts', [0, 50, 100])
    buffer_size = trial.suggest_categorical('buffer_size', [40_000, 50_000, 60_000])
    train_freq = trial.suggest_categorical('train_freq', [4,6,8])
    gradient_steps = trial.suggest_categorical('gradient_steps', [-1])
    exploration_fraction = trial.suggest_float('exploration_fraction', 0.1, 0.2)
    exploration_final_eps = trial.suggest_float('exploration_final_eps', 0.05, 0.15)

    net_arch = [layer_size, layer_size]
    policy_kwargs = dict(net_arch=net_arch)
    
    model = DQN(
        "MlpPolicy", 
        trial_env, 
        learning_rate=learning_rate, 
        gamma=gamma,
        batch_size=batch_size,
        exploration_fraction=exploration_fraction,
        exploration_final_eps=exploration_final_eps,
        learning_starts=learning_starts,
        buffer_size=buffer_size,
        train_freq=train_freq,
        gradient_steps=gradient_steps,
        policy_kwargs=policy_kwargs,
        verbose=0
    )
    
    model.learn(total_timesteps=15000)
    
    mean_reward, _ = evaluate_policy(model, trial_env, n_eval_episodes=50)
    trial_env.close()
    return mean_reward

In [ ]:
LOG_DIR = "out/logs/"
os.makedirs(LOG_DIR, exist_ok=True)
STORAGE_PATH = "sqlite:///out/my_study.db"
STUDY_NAME = "dqn-ll-optimization"
NUM_TRIALS_TO_RUN = 100

study = optuna.create_study(
    direction="maximize",
    study_name=STUDY_NAME,
    storage=STORAGE_PATH,
    load_if_exists=True
)

best_trial = study.best_trial
if best_trial:
    print(f"  Value (Mean Reward): {best_trial.value:.2f}")
    print("  Params: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
        
    best_params = best_trial.params.copy()
    final_layer_size = best_params.pop('layer_size')
    final_policy_kwargs = dict(net_arch=[final_layer_size, final_layer_size])
    
    final_env = gym.make("LunarLander-v3")
    final_log_path = os.path.join(LOG_DIR, "final_model_logs")
    final_env = Monitor(final_env, final_log_path)
    
    final_model = DQN("MlpPolicy", final_env, policy_kwargs=final_policy_kwargs,
                      **best_params, verbose=0)
    
    final_model.learn(total_timesteps=50000)
    final_model.save("out/best_lunar_model")
    
    eval_env = Monitor(gym.make("LunarLander-v3"), 'out/logs/final_model_eval')
    mean_reward, std_reward = evaluate_policy(final_model, eval_env, n_eval_episodes=100)
    print(f"Final Model: Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")
    eval_env.close()
    

(exempel) output:

  Value (Mean Reward): 18.01

  Params: <br>
  <code>learning_rate: 0.0007525695954787247 <br>
    gamma: 0.9831604862288565 <br>
    layer_size: 256 <br>
    batch_size: 128<br>
    learning_starts: 0<br>
    buffer_size: 40000<br>
    train_freq: 8<br>
    gradient_steps: -1 <br>
    exploration_fraction: 0.17139725460556557 <br>
    exploration_final_eps: 0.056871603608688294 <br> </code>

Final Model: Mean reward = -89.55 +/- 129.90

Körde studien ett antal gånger och testade en del av de bättre resultaten genom att träna modellen lite längre med hyperparametrarna. Inga av de testade producerade direkt bra resultat.

DQN behövde också allmänt några hundra tusen steg i miljön för att visa resultat man kunde dra några slutsatser av, men att köra en hyperparameteroptimations studie var varje kombination testas minst runt hundra tusen steg för att på riktigt få en uppfatning om kombinationen skulle inte ha varit görbart.

#### Training results

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [140]:
ppo_data = np.load('logs/ppo/evaluations.npz')
ppo_timesteps, ppo_results = ppo_data["timesteps"], ppo_data["results"]

ppo_mean_rewards = ppo_results.mean(axis=1)
ppo_std_rewards = ppo_results.std(axis=1)

In [141]:
fig = go.Figure()
fig.add_scatter(x=ppo_timesteps, y=ppo_mean_rewards, name='Mean rewards', mode='lines')
fig.add_scatter(x=ppo_timesteps, y=ppo_mean_rewards - ppo_std_rewards, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'),
                         fillcolor='rgba(255,0,0,0.2)', showlegend=False, name='std')
fig.add_scatter(x=ppo_timesteps, y=ppo_mean_rewards + ppo_std_rewards, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'),
                         fillcolor='rgba(255,0,0,0.2)', name='std')
fig.update_layout(title_text='PPO on LunarLander-v3', title_subtitle_text='Evaluation under training', 
                  xaxis=dict(title=dict(text='timesteps')), yaxis=dict(title=dict(text='rewards')),
                  autosize=False, width=1000, height=500)
fig.show()

[plot](plots/ppo_train.png)

Det kunde antagligen ha räckt att träna PPO i ca. 1.5M timesteps, men som resultaten av träningen visar harmades inte modellen av lite överträning

In [74]:
data = np.load('logs/dqn/evaluations.npz')
timesteps, results = data["timesteps"], data["results"]

mean_rewards = results.mean(axis=1)
std_rewards = results.std(axis=1)

In [138]:
fig = go.Figure()
fig.add_scatter(x=timesteps, y=mean_rewards, name='Mean rewards', mode='lines')
fig.add_scatter(x=timesteps, y=mean_rewards - std_rewards, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'),
                         fillcolor='rgba(255,0,0,0.2)', showlegend=False, name='std')
fig.add_scatter(x=timesteps, y=mean_rewards + std_rewards, fill='tonexty', mode='lines', line=dict(color='rgba(0,0,0,0)'),
                         fillcolor='rgba(255,0,0,0.2)', name='std')
fig.update_layout(title_text='DQN on LunarLander-v3', title_subtitle_text='Evaluation under training', 
                  xaxis=dict(title=dict(text='timesteps')), yaxis=dict(title=dict(text='rewards')),
                  autosize=False, width=1000, height=500)
fig.show()

[plot](plots/dqn_train.png)

Man kan se att i slutet av träningen börjar DQN prestera en aning sämre. Detta kan delvis bero på slumpen, då startförhållandena i LunarLander är delvis slumpmässiga, men också vara ett tecken på överträning. Dryga 800t timesteps kunde möjligen ha räckt.

DQN behövde mindre träning än PPO för att nå liknande resultat under träningen och producera en modell som lyckades i miljön under testning. 

I början av träningen lyckas PPO få värre poäng än DQN, detta kan bero på att PPO börjar inlärningnen genast utan någon typ av uppfattning av miljön (dvs. utför slumpmässiga handlingar) medans DQN har haft en chans att observera miljön (`learning_starts=100`) och har lite uppfattning om miljön och kan då utföra en aning bättre handlingar. Detta var något som hände konsekvent i träning av modellerna.

DQN tycks vara lite ostabilare under träningen, och har lite högre varians i resultaten den når.

#### Testing

>Testning i slumpmässigt prodcerade miljöer

In [76]:
env = gym.make('LunarLander-v3')

In [77]:
dqn_best = DQN.load('models/dqn_best/best_model.zip')

In [78]:
rewards_dqn = []
for i in range(50):
    print(f"*** Episode {i} ***")

    done = False
    steps = 0
    episode_r = 0
    state, _ = env.reset()

    while not done:
      action, _ = dqn_best.predict(state, deterministic=True)
      state, reward, done, truncated, info = env.step(action)
      episode_r += reward
      steps += 1

      if truncated:
        break
    
    rewards_dqn.append(episode_r)
    print(f"Steps taken: {steps},   reward: {episode_r}")

*** Episode 0 ***
Steps taken: 254,   reward: 283.9438358168338
*** Episode 1 ***
Steps taken: 252,   reward: 261.7468444378556
*** Episode 2 ***
Steps taken: 292,   reward: 248.9280270021495
*** Episode 3 ***
Steps taken: 295,   reward: 260.81622538659155
*** Episode 4 ***
Steps taken: 262,   reward: 299.54232781889243
*** Episode 5 ***
Steps taken: 324,   reward: 261.8976829201047
*** Episode 6 ***
Steps taken: 203,   reward: 312.49710891156735
*** Episode 7 ***
Steps taken: 340,   reward: 230.01567993404296
*** Episode 8 ***
Steps taken: 309,   reward: 253.2566614703841
*** Episode 9 ***
Steps taken: 214,   reward: 254.33908961454677
*** Episode 10 ***
Steps taken: 216,   reward: 277.9815746181656
*** Episode 11 ***
Steps taken: 261,   reward: 289.89547698454624
*** Episode 12 ***
Steps taken: 225,   reward: 271.10750797952915
*** Episode 13 ***
Steps taken: 268,   reward: 241.60447337245319
*** Episode 14 ***
Steps taken: 352,   reward: 259.46385329126247
*** Episode 15 ***
Steps t

In [15]:
ppo_best = PPO.load('models/ppo_best/best_model.zip')

In [16]:
rewards_ppo = []
for i in range(50):
    print(f"*** Episode {i} ***")

    done = False
    steps = 0
    episode_r = 0
    state, _ = env.reset()

    while not done:
      action, _ = ppo_best.predict(state, deterministic=True)
      state, reward, done, truncated, info = env.step(action)
      episode_r += reward
      steps += 1

      if truncated:
        break

    rewards_ppo.append(episode_r)
    print(f"Steps taken: {steps},   reward: {episode_r}")

*** Episode 0 ***
Steps taken: 274,   reward: 272.43008999292186
*** Episode 1 ***
Steps taken: 298,   reward: 270.29975095497986
*** Episode 2 ***
Steps taken: 293,   reward: 272.21245579125554
*** Episode 3 ***
Steps taken: 314,   reward: 274.23299708667173
*** Episode 4 ***
Steps taken: 270,   reward: 301.0063870260986
*** Episode 5 ***
Steps taken: 301,   reward: 257.6782458816821
*** Episode 6 ***
Steps taken: 295,   reward: 282.61092958206143
*** Episode 7 ***
Steps taken: 288,   reward: 299.0227112542998
*** Episode 8 ***
Steps taken: 289,   reward: 218.2805788696512
*** Episode 9 ***
Steps taken: 290,   reward: 266.2225814988759
*** Episode 10 ***
Steps taken: 293,   reward: 279.3627294019717
*** Episode 11 ***
Steps taken: 285,   reward: 270.4340480017354
*** Episode 12 ***
Steps taken: 291,   reward: 239.65251081196962
*** Episode 13 ***
Steps taken: 299,   reward: 298.7963771423192
*** Episode 14 ***
Steps taken: 254,   reward: 295.413971615735
*** Episode 15 ***
Steps taken

>Policy evaluering

In [27]:
mean_reward, std_dev = evaluate_policy(ppo_best, env, n_eval_episodes=20)
print(f"Mean reward: {mean_reward}, std dev: {std_dev:.4f}")

Mean reward: 264.378148372929, std dev: 18.4117


In [120]:
mean_reward, std_dev = evaluate_policy(dqn_best, env, n_eval_episodes=20)
print(f"Mean reward: {mean_reward}, std dev: {std_dev:.4f}")

Mean reward: 285.4689241308515, std dev: 18.7179


In [ ]:
x = list(range(1,len(rewards_ppo)))
fig = go.Figure()
fig.add_scatter(x=x, y=rewards_ppo, name='PPO', mode='lines')
fig.add_scatter(x=x, y=rewards_dqn, name='DQN', mode='lines')
fig.update_layout(title_text='PPO vs DQN', xaxis=dict(title=dict(text='episodes')), 
                  yaxis=dict(title=dict(text='mean rewards')), autosize=False, width=1000, height=500)
fig.show()

[plot](plots/testing.png)

>Testning med identiska miljöer (seed för att skapa samma startmiljö för båda)

In [122]:
rewards_dqn_eq = []
rewards_ppo_eq = []

steps_dqn_eq = []
steps_ppo_eq = []
for _ in range(20):
    seed = np.random.randint(20,200)

    done = False
    steps = 0
    episode_r = 0
    env = gym.make("LunarLander-v3")
    state, _ = env.reset(seed=seed)

    while not done:
      action, _ = dqn_best.predict(state, deterministic=True)
      state, reward, done, truncated, info = env.step(action)
      episode_r += reward
      steps += 1

      if truncated:
        break
    
    rewards_dqn_eq.append(episode_r)
    steps_dqn_eq.append(steps)

    done = False
    steps = 0
    episode_r = 0
    env = gym.make("LunarLander-v3")
    state, _ = env.reset(seed=seed)

    while not done:
      action, _ = ppo_best.predict(state, deterministic=True)
      state, reward, done, truncated, info = env.step(action)
      episode_r += reward
      steps += 1

      if truncated:
        break

    rewards_ppo_eq.append(episode_r)
    steps_ppo_eq.append(steps)

In [142]:
fig = go.Figure().set_subplots(2,1)

x = list(range(1,len(rewards_ppo_eq)))
fig.add_scatter(x=x, y=rewards_ppo_eq, name='PPO rewards', mode='lines', row=1, col=1)
fig.add_scatter(x=x, y=rewards_dqn_eq, name='DQN rewards', mode='lines', row=1, col=1)

x = list(range(1,len(steps_ppo_eq)))
fig.add_scatter(x=x, y=steps_ppo_eq, name='PPO steps', mode='lines', row=2, col=1)
fig.add_scatter(x=x, y=steps_dqn_eq, name='DQN steps', mode='lines', row=2, col=1)

fig.update_layout(title_text='PPO vs DQN', title_subtitle_text='Testing in identical environments', 
                  xaxis2=dict(title=dict(text='episodes')), yaxis1=dict(title=dict(text='rewards')), 
                  yaxis2=dict(title=dict(text='steps')), autosize=False, width=1000, height=600)
fig.show()

[plot](plots/testing_eq.png)

Båda tränade modellerna utför miljön bra, DQN når något högre poäng än PPO.

I testning i identiska miljöer når båda agenterna väldigt lika poäng, DQN är dock i nästan alla episoder just lite bättre. Antalet steg agenterna utför i episoderna är intressantare, båda följer liknande mönster men DQN utför alltid mindre antal steg.

Episod 7 & 10 är intressanta – PPO får just lite fler poäng men tar ändå fler steg än DQN. Dessa är också de episoder var antalet steg är närmast varandra. Möjligen har DQN utfört mer kostsamma handlingar än PPO i dessa episoder.

På basis av dessa resultat skulle man kunna dra slutsatsen att denna DQN modell utför miljön bättre än denna PPO modell. DQN utför mindre steg för högre poäng, vilket på basis av miljöns belöningssystem, skulle påvisa att DQN utför landningen på ett stabilare sätt.

>I LunarLander är det helt acceptabelt att ha variation i poängen mellan episoder då startförhållandena är delvis slumpmässiga vilket gör att landningsättet (och då poängen) behöver variera episod till episod.